In [43]:
import rasterio
from osgeo import gdal
from pathlib import Path
import shutil
from tqdm.notebook import tqdm
from PIL import Image
import os

In [14]:
ROOT = Path()
DATA = ROOT / "DATA"
TILES = DATA / "TILES"
PRE_DIR = TILES / "PRE_EVENT"
POST_DIR = TILES / "POST_EVENT"

In [8]:
PRE_IMG = DATA / "Pre_Event_San_Juan.tif"
POST_IMG = DATA / "Post_Event_San_Juan.tif"

# Tiling Images

In [41]:
def generate_tiles(
    input_file: str,
    output_dir: str,
    x_grid_size: int,
    y_grid_size: int,
):
    # Cleaning directory so we can rerun without duplicates
    shutil.rmtree(output_dir, ignore_errors=True)
    
    # Image size & band counts
    ds = gdal.Open(input_file)
    width = ds.RasterXSize
    height = ds.RasterYSize
    bands = ds.RasterCount
    
    # // -> Floor division
    num_tiles_x = (width // x_grid_size)
    num_tiles_y = (width // y_grid_size)
    
    print(f"Tile counts (x, y): {num_tiles_x, num_tiles_y}")
    
    # Create output directory
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Tiling
    for x in tqdm(range(num_tiles_x)):
        for y in range(num_tiles_y):
            x_offset = x * x_grid_size
            y_offset = y * y_grid_size
            
            tile_width = min(x_grid_size, width - x_offset)
            tile_height = min(y_grid_size, height - y_offset)
            
            tile = []
            for band in range(1, bands+1):
                tile_data = (
                    ds.GetRasterBand(band)
                    .ReadAsArray(
                        x_offset,
                        y_offset,
                        tile_width,
                        tile_height
                    )
                )
                tile.append(tile_data)
            
            # Output filename
            output_name = output_dir / f"tile_{x}_{y}.tif"
            
            # Output TIFF
            driver = gdal.GetDriverByName("GTiff")
            options = ['COMPRESS=DEFLATE', 'PREDICTOR=2', 'TILED=YES']
            out_ds = driver.Create(
                output_name.as_posix(),
                tile_width,
                tile_height,
                bands,
                ds.GetRasterBand(1).DataType,
                options=options
            )
            
            # Set Geotransform
            geotransform = list(ds.GetGeoTransform())
            geotransform[0] = geotransform[0] + x_offset * geotransform[1]
            geotransform[3] = geotransform[3] + x_offset * geotransform[5]
            out_ds.SetGeoTransform(tuple(geotransform))
            
            # Projection
            out_ds.SetProjection(ds.GetProjection())
            
            for band in range(1, bands+1):
                out_band = out_ds.GetRasterBand(band)
                out_band.WriteArray(tile[band-1])
            
            # Close output file
            out_ds = None
        

In [42]:
for source_file, target in (
    (PRE_IMG.as_posix(), PRE_DIR),
    (POST_IMG.as_posix(), POST_DIR),
):
    print(f"Processing {source_file} ...")
    generate_tiles(
        source_file, 
        target,
        512,
        512
    )
    print()

Tile counts (x, y): (74, 74)


  0%|          | 0/74 [00:00<?, ?it/s]

Tile counts (x, y): (74, 74)


  0%|          | 0/74 [00:00<?, ?it/s]

# Convert TIFF to JPEG

In [85]:
def convert_tiff2jpeg(input_dir, output_dir):
    print(f"Start processing for {input_dir.parts[-1]}")
    shutil.rmtree(output_dir, ignore_errors=True)
    
    output_dir.mkdir(exist_ok=True, parents=True)
    
    for filename in tqdm(list(input_dir.glob("*.tif"))):
        img = Image.open(filename)
        
        if img.mode != "RGB":
            img = img.convert("RGB")
        
        output_name = output_dir / (filename.with_suffix(".jpg"))
        
        img.save(output_name, "JPEG")
    print("Processing done\n")


In [59]:
POST_JPEG = TILES / "POST_JPEG"
PRE_JPEG = TILES / "PRE_JPEG"

for source, target in (
    (POST_DIR, POST_JPEG),
    (PRE_DIR, PRE_JPEG),
):
    convert_tiff2jpeg(
        source,
        target
    )

Start processing for POST_EVENT


  0%|          | 0/5476 [00:00<?, ?it/s]

Processing done

Start processing for PRE_EVENT


  0%|          | 0/5476 [00:00<?, ?it/s]

Processing done



In [98]:
def rename_files(path, prefix: str):
    files = path.glob("*.jpg")
    for i, file in tqdm(enumerate(files)):
        extension = file.suffix
        new_name = f"{prefix}_{i:04}{extension}"
        
        old_filepath = file
        new_filepath = path / new_name
        
        os.rename(old_filepath, new_filepath)

for path, prefix in (
    (POST_JPEG, "Post_Event"),
    (PRE_JPEG, "Pre_Event")
):
    rename_files(path, prefix)

0it [00:00, ?it/s]

0it [00:00, ?it/s]